In [1]:
import pandas as pd
import yfinance
import numpy as np
import pandas_ta as pta

In [2]:
ticker = yfinance.Ticker("INFY.NS")
df = ticker.history(period = '5y')
df.drop(columns=['Dividends','Stock Splits'],inplace=True)

In [5]:
help(pta.ema)

Help on function ema in module pandas_ta.overlap.ema:

ema(close, length=None, talib=None, offset=None, **kwargs)
    Exponential Moving Average (EMA)
    
    The Exponential Moving Average is more responsive moving average compared to the
    Simple Moving Average (SMA).  The weights are determined by alpha which is
    proportional to it's length.  There are several different methods of calculating
    EMA.  One method uses just the standard definition of EMA and another uses the
    SMA to generate the initial value for the rest of the calculation.
    
    Sources:
        https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_averages
        https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp
    
    Calculation:
        Default Inputs:
            length=10, adjust=False, sma=True
        if sma:
            sma_nth = close[0:length].sum() / length
            close[:length - 1] = n

In [21]:
#calculate SMA
df['SMA5']=pta.sma(df['Close'],length=5)
df['SMA10']=pta.sma(df['Close'],length=10)
df['SMA20']=pta.sma(df['Close'],length=20)
df['SMA50']=pta.sma(df['Close'],length=50)
df['SMA100']=pta.sma(df['Close'],length=100)
df['SMA200']=pta.sma(df['Close'],length=200)

#Calculate MACD and Signal line
df['MACD_LINE']= pta.macd(df['Close'])['MACD_12_26_9']
df['MACD_SIGNAL']= pta.macd(df['Close'])['MACDs_12_26_9']

#Calculate RSI
df['RSI']=pta.rsi(df['Close'], length = 14)

#Calculate Stochastic(20,3)
df['STOCHK'] = pta.stoch(df['High'],df['Low'],df['Close'],k=20, d=3).STOCHk_20_3_3
df['STOCHD'] = pta.stoch(df['High'],df['Low'],df['Close'],k=20, d=3).STOCHd_20_3_3

#Calculate ROC(20)
df['ROC'] = pta.roc(df['Close'],length=20)

#Calculate CCI(20)
df['CCI'] = pta.cci(df['High'],df['Low'],df['Close'],length=20)

#Calculate Williamson%R(14)
df['WILLR'] = pta.willr(df['High'],df['Low'],df['Close'],length=14)

#Calculate MFI(14)
df['MFI'] = pta.mfi(df['High'],df['Low'],df['Close'],df['Volume'], length=14)

#Calculate ADX(14)
df['ADX'] = pta.adx(df['High'],df['Low'],df['Close'])['ADX_14']

#Calculate ATR(14)
df['ATR'] = pta.atr(df['High'],df['Low'],df['Close'],length=14)
df['ATR'] = pta.atr(df['High'],df['Low'],df['Close'],length=14)

#Calculate Bollinger Band(20,2)
df['BBL'] = pta.bbands(df['Close'],length=20)['BBL_20_2.0']
df['BBM'] = pta.bbands(df['Close'],length=20)['BBM_20_2.0']
df['BBH'] = pta.bbands(df['Close'],length=20)['BBU_20_2.0']

In [ ]:
import pandas_ta as pta
help(pta.donchian)

In [23]:
df['S_M5']=0
df.loc[df['Close'] > df['SMA5'], 'S_M5'] = 1

df['S_M10']=0
df.loc[df['Close'] > df['SMA10'], 'S_M10'] = 1

df['S_M20']=0
df.loc[df['Close'] > df['SMA20'], 'S_M20'] = 1

df['S_M50']=0
df.loc[df['Close'] > df['SMA50'], 'S_M50'] = 1

df['S_M100']=0
df.loc[df['Close'] > df['SMA100'], 'S_M100'] = 1

df['S_M200']=0
df.loc[df['Close'] > df['SMA200'], 'S_M200'] = 1

In [24]:
df['S_MC_5_20']=0
df.loc[df['SMA5'] > df['SMA20'], 'S_MC_5_20'] = 1

df['S_MC_20_50']=0
df.loc[df['SMA20'] > df['SMA50'], 'S_MC_20_50'] = 1

df['S_MC_50_200']=0
df.loc[df['SMA50'] > df['SMA200'], 'S_MC_50_200'] = 1

df['VOL20']=pta.sma(df['Volume'],length=20)
df['VOL5']=pta.sma(df['Volume'],length=5)

df['TARGET'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)

df['H-L'] = df['High']-df['Low']
df['C-O']=df['Close']-df['Open']

In [25]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'SMA5', 'SMA10', 'SMA20',
       'SMA50', 'SMA100', 'SMA200', 'MACD_LINE', 'MACD_SIGNAL', 'RSI',
       'STOCHK', 'STOCHD', 'ROC', 'CCI', 'WILLR', 'MFI', 'ADX', 'ATR', 'BBL',
       'BBM', 'BBH', 'S_M5', 'S_M10', 'S_M20', 'S_M50', 'S_M100', 'S_M200',
       'S_MC_5_20', 'S_MC_20_50', 'S_MC_50_200', 'VOL20', 'VOL5', 'TARGET',
       'H-L', 'C-O'],
      dtype='object')

In [25]:
df_fundamental = pd.DataFrame.from_dict(ticker.get_info(), orient='index',columns=['value'])

In [26]:
df_fundamental.reset_index(level=0, inplace=True)

In [28]:
df_fundamental

,index,value
0,zip,560100
1,sector,Technology
2,fullTimeEmployees,292067
3,longBusinessSummary,"Infosys Limited, together with its subsidiarie..."
4,city,Bengaluru
...,...,...
148,bidSize,0
149,dayHigh,1749
150,regularMarketPrice,1731.3
151,preMarketPrice,None


In [27]:
df_fundamental[df_fundamental['index']=='sector']['value'].to_list()[0]

'Technology'

In [146]:
ticker.earnings


,Revenue,Earnings
Year,,
2018,10939000000,2486000000
2019,11799000000,2199000000
2020,12780000000,2331000000
2021,13561000000,2613000000


In [151]:
ticker.financials

,2021-03-31,2020-03-31,2019-03-31,2018-03-31
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,3596000000.0,3095000000.0,3003000000.0,3143000000.0
Minority Interest,60000000.0,55000000.0,9000000.0,10000000.0
Net Income,2613000000.0,2331000000.0,2199000000.0,2486000000.0
Selling General Administrative,1281000000.0,1353000000.0,1281000000.0,1151000000.0
Gross Profit,4608000000.0,4077000000.0,3978000000.0,3810000000.0
Ebit,3327000000.0,2724000000.0,2697000000.0,2659000000.0
Operating Income,3327000000.0,2724000000.0,2697000000.0,2659000000.0
Other Operating Expenses,NaN,NaN,13120000000.0,10020000000.0


In [150]:
ticker.get_actions(all)[["Dividends"]]

,Dividends
Date,
2017-06-01,7.375
2017-10-31,6.500
2018-06-14,5.000
2018-09-04,0.000
2018-10-25,7.000
2019-01-24,4.000
2019-06-13,10.500
2019-10-22,8.000
2019-10-23,8.000


In [30]:
ticker.info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'fax', 'industry', 'address2', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskR

In [ ]:
 'fiftyTwoWeekHigh': 1953.9
'fiftyTwoWeekLow': 1244.75
'logo_url': 'https://logo.clearbit.com/infosys.com'
    'marketCap': 7256189960192
        'currency': 'INR'
            'trailingEps': 51.459,
                'trailingPE': 33.64426,
                    'beta': 0.663233
                        'exchange': 'NSI',
                            'recommendationKey': 'buy',
                                'sector': 'Technology',
                                    'industry': 'Information Technology Services',
                                        'ebitda': 4084000000,

In [64]:

df_shareinfo

""


In [74]:
df_shareinfo = pd.DataFrame()
df_shareinfo = pd.concat([pd.DataFrame([pd.Series(ticker.info.values())]), df_shareinfo], ignore_index=False)
df_shareinfo.columns =list(ticker.info.keys())
df_shareinfo

,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,country,companyOfficers,website,maxAge,...,fiveYearAvgDividendYield,fiftyTwoWeekLow,bid,tradeable,dividendYield,bidSize,dayHigh,regularMarketPrice,preMarketPrice,logo_url
0,560100,Technology,292067,"Infosys Limited, together with its subsidiarie...",Bengaluru,91 80 2852 0261,India,[],https://www.infosys.com,1,...,2.26,1244.75,1731.3,False,0.0173,0,1749,1731.3,None,https://logo.clearbit.com/infosys.com


In [76]:
list(df_shareinfo.columns)

['zip',
 'sector',
 'fullTimeEmployees',
 'longBusinessSummary',
 'city',
 'phone',
 'country',
 'companyOfficers',
 'website',
 'maxAge',
 'address1',
 'fax',
 'industry',
 'address2',
 'ebitdaMargins',
 'profitMargins',
 'grossMargins',
 'operatingCashflow',
 'revenueGrowth',
 'operatingMargins',
 'ebitda',
 'targetLowPrice',
 'recommendationKey',
 'grossProfits',
 'freeCashflow',
 'targetMedianPrice',
 'currentPrice',
 'earningsGrowth',
 'currentRatio',
 'returnOnAssets',
 'numberOfAnalystOpinions',
 'targetMeanPrice',
 'debtToEquity',
 'returnOnEquity',
 'targetHighPrice',
 'totalCash',
 'totalDebt',
 'totalRevenue',
 'totalCashPerShare',
 'financialCurrency',
 'revenuePerShare',
 'quickRatio',
 'recommendationMean',
 'exchange',
 'shortName',
 'longName',
 'exchangeTimezoneName',
 'exchangeTimezoneShortName',
 'isEsgPopulated',
 'gmtOffSetMilliseconds',
 'quoteType',
 'symbol',
 'messageBoardId',
 'market',
 'annualHoldingsTurnover',
 'enterpriseToRevenue',
 'beta3Year',
 'enterpr

In [77]:

ticker.actions

,Dividends,Stock Splits
Date,,
2017-06-01,7.375,0.0
2017-10-31,6.500,0.0
2018-06-14,5.000,0.0
2018-09-04,0.000,2.0
2018-10-25,7.000,0.0
2019-01-24,4.000,0.0
2019-06-13,10.500,0.0
2019-10-22,8.000,0.0
2019-10-23,8.000,0.0


In [79]:
ticker.balance_sheet

,2021-03-31,2020-03-31,2019-03-31,2018-03-31
Intangible Assets,2.830000e+08,2.510000e+08,1.000000e+08,3.800000e+07
Capital Surplus,3.590000e+08,3.050000e+08,2.770000e+08,2.470000e+08
Total Liab,4.323000e+09,3.559000e+09,2.852000e+09,2.295000e+09
Total Stockholder Equity,1.044200e+10,8.646000e+09,9.391000e+09,9.960000e+09
Minority Interest,6.000000e+07,5.500000e+07,9.000000e+06,1.000000e+07
Deferred Long Term Liab,2.000000e+07,1.300000e+07,4.000000e+07,4.000000e+07
Other Current Liab,1.362000e+09,1.138000e+09,1.368000e+09,1.078000e+09
Total Assets,1.482500e+10,1.226000e+10,1.225200e+10,1.225500e+10
Common Stock,3.320000e+08,3.320000e+08,3.390000e+08,1.900000e+08
Other Current Assets,6.530000e+08,5.060000e+08,5.850000e+08,7.360000e+08


In [80]:
ticker.earnings

,Revenue,Earnings
Year,,
2018,10939000000,2486000000
2019,11799000000,2199000000
2020,12780000000,2331000000
2021,13561000000,2613000000


In [81]:
ticker.financials

,2021-03-31,2020-03-31,2019-03-31,2018-03-31
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,3596000000.0,3095000000.0,3003000000.0,3143000000.0
Minority Interest,60000000.0,55000000.0,9000000.0,10000000.0
Net Income,2613000000.0,2331000000.0,2199000000.0,2486000000.0
Selling General Administrative,1281000000.0,1353000000.0,1281000000.0,1151000000.0
Gross Profit,4608000000.0,4077000000.0,3978000000.0,3810000000.0
Ebit,3327000000.0,2724000000.0,2697000000.0,2659000000.0
Operating Income,3327000000.0,2724000000.0,2697000000.0,2659000000.0
Other Operating Expenses,NaN,NaN,13120000000.0,10020000000.0


In [83]:
df_news = ticker.get_news()

In [88]:
df_news = pd.DataFrame(df_news)

In [89]:
df_news

,uuid,title,publisher,link,providerPublishTime,type
0,abafc87b-c3e2-3b0d-b9b8-848c609e6322,Nestle proposes Apple CFO for election to its ...,Reuters,https://finance.yahoo.com/news/nestle-proposes...,1645080369,STORY
1,3c460b19-6891-3b11-aaa0-ae501968024d,Germany set to end most COVID-19 restrictions ...,MarketWatch,https://finance.yahoo.com/m/3c460b19-6891-3b11...,1645080300,STORY
2,4cfdb3b1-edf7-3eb5-84e3-1f028b7414c2,Global K-12 Makerspace Materials Market 2022 N...,GlobeNewswire,https://finance.yahoo.com/news/global-k-12-mak...,1645080300,STORY
3,a6949e98-f1c5-3ce8-8079-89bf61d76d04,Kering 2021 FY Results: EXCELLENT 2021 PERFORM...,GlobeNewswire,https://finance.yahoo.com/news/kering-2021-fy-...,1645080300,STORY
4,cfeed42b-ad43-3bf0-a3d2-1ac425785b81,Kojamo plc's Financial Statements Release 1 Ja...,PR Newswire,https://finance.yahoo.com/news/kojamo-plcs-fin...,1645080240,STORY
5,fa9ee7e3-6265-38a4-92a7-b222452a9dd1,"Satellite group Eutelsat narrows, postpones fi...",Reuters,https://finance.yahoo.com/news/satellite-group...,1645080039,STORY
6,63a6e5ba-ed4b-32eb-999f-3d78ca63a383,Vistin Pharma ASA: Fourth quarter 2021 financi...,GlobeNewswire,https://finance.yahoo.com/news/vistin-pharma-a...,1645080000,STORY
7,c44f004c-8ae9-39ff-8008-3dff05cbae34,Honda says domestic output capacity reduced un...,Reuters,https://finance.yahoo.com/news/honda-says-dome...,1645079944,STORY


In [92]:
pd.to_datetime(df_news['providerPublishTime'][0])

Timestamp('1970-01-01 00:00:01.645080369')

In [103]:
df_news['providerPublishTime'] = pd.to_datetime(df_news['providerPublishTime'], unit='s')

In [104]:
df_news['providerPublishTime'] 

0   2022-02-17 06:46:09
1   2022-02-17 06:45:00
2   2022-02-17 06:45:00
3   2022-02-17 06:45:00
4   2022-02-17 06:44:00
5   2022-02-17 06:40:39
6   2022-02-17 06:40:00
7   2022-02-17 06:39:04
Name: providerPublishTime, dtype: datetime64[ns]

In [108]:
df_news['title']

0    Nestle proposes Apple CFO for election to its ...
1    Germany set to end most COVID-19 restrictions ...
2    Global K-12 Makerspace Materials Market 2022 N...
3    Kering 2021 FY Results: EXCELLENT 2021 PERFORM...
4    Kojamo plc's Financial Statements Release 1 Ja...
5    Satellite group Eutelsat narrows, postpones fi...
6    Vistin Pharma ASA: Fourth quarter 2021 financi...
7    Honda says domestic output capacity reduced un...
Name: title, dtype: object

In [113]:
ticker.quarterly_earnings

,Revenue,Earnings
Quarter,,
1Q2021,3613000000,697000000
2Q2021,3782000000,704000000
3Q2021,3998000000,733000000
4Q2021,4250000000,774000000


# Twitter Analysis

In [3]:
import pandas as pd
data = pd.read_csv('/Users/vivek/code/vivekjainmaiet/stockanalysis/2022-02-17.csv')

In [6]:
data['sentiment'].mean()

0.10692345899470901

In [7]:
!pip install finvizfinance

     |████████████████████████████████| 42 kB 1.8 MB/s             
     |████████████████████████████████| 51 kB 2.2 MB/s             
You should consider upgrading via the '/Users/vivek/.pyenv/versions/3.8.12/envs/lewagon/bin/python3.8 -m pip install --upgrade pip' command.


In [8]:
import finvizfinance 
import finvizfinance.screener 
from finvizfinance.quote import finvizfinance

def get_fin_news(ticker = 'AAPL'):
    # obtain news per ticker
    stock = finvizfinance(ticker)
    news = stock.ticker_news()
    news['Ticker'] = ticker
    news['Date']= pd.to_datetime(news['Date'])
    # save the news
    # news.to_csv('/Users/yingxu/stockanalysis/notebooks/news')
    return news

In [10]:
get_fin_news()

,Date,Title,Link,Ticker
0,2022-02-19 07:35:00,Dow Jones Futures: Biden Says Putin Has Decide...,https://www.investors.com/market-trend/stock-m...,AAPL
1,2022-02-19 07:30:00,6 Anti-Trust Laws That Smart Investors Should ...,https://www.fool.com/investing/2022/02/19/6-an...,AAPL
2,2022-02-18 19:04:00,Dow Jones Futures: Market Rally Nearing Lows O...,https://www.investors.com/market-trend/stock-m...,AAPL
3,2022-02-18 17:14:00,Apple store unions could jeopardize its 'carin...,https://finance.yahoo.com/news/apple-could-jeo...,AAPL
4,2022-02-18 16:38:00,ISS supports Apple shareholder proposal on for...,https://finance.yahoo.com/news/iss-supports-ap...,AAPL
...,...,...,...,...
95,2022-02-15 11:50:00,EU deal on single mobile charging port by year...,https://finance.yahoo.com/news/eu-deal-single-...,AAPL
96,2022-02-15 11:38:00,"Disney stock extends earnings relief rally, 'd...",https://finance.yahoo.com/news/disney-bank-of-...,AAPL
97,2022-02-15 11:38:00,Apple Stock Is Beating the Rest of Big Tech. W...,https://www.barrons.com/articles/apple-stock-p...,AAPL
98,2022-02-15 11:16:00,Best Growth Stocks to Buy for February 15th,https://finance.yahoo.com/news/best-growth-sto...,AAPL
